In [ ]:
# Imports
using Gurobi
using JuMP
using Random
using LaTeXStrings
using Distributed
include("fit.jl");
include("bin_to_leaves.jl");

In [ ]:
# Generating data
a = 1; b = 2; 
xβnge(1,5, step=0.1);
y = range(1,5, step=0.1);
X = zeros(size(x,1)*size(y,1), 2)
for i=1:size(x,1)
    for j=1:size(y,1)
        X[(i-1)*size(y,1)+j, :] = [x[i], y[j]];
    end
end
# Y = X[:,1].^2 - 0.05*(X[:,1] .- 2) .^3 .*X[:,2] - 4*X[:,1].*(X[:,2] .- 1)# + exp.(X[:,1]./X[:,2]);
Y = max.(0, X[:,1].-2 - 0.1*X[:,1].*X[:,2].^2, X[:,2].-2);

In [ ]:
# Plotting data
using Plots
plt = Plots.plot(X[:,2], X[:,1], Y[:,1], seriestype=:scatter, markersize = 2, camera=(10,50), legend=false);
display(plt)

In [ ]:
model = JuMP.Model(solver=GurobiSolver())
@variable(model, a >= 0)

In [ ]:
@constraint(model, a >=5)
@objective(model, Min, a)

In [ ]:
JuMP.solve(model)

In [ ]:
# Trees
# First with parallel splits
include("augment.jl");
# X = hcat(X, X[:,1].^2, X[:,2].^2, X[:,1].*X[:,2])#, X[:,1].*X[:,2].^2, X[:,1].^2 .*X[:,2], X[:,1].^3, X[:,2].^3);
nX, means, vars = normalize(X);
lnr = IAI.OptimalTreeRegressor(random_seed=15, max_depth=3, cp=1e-8, minbucket=0.02, 
    regression_sparsity=:all, split_features=[1,2], hyperplane_config=(sparsity=1,),
    regression_lambda=0.00001, regression_weighted_betas=true, fast_num_support_restarts=3)
# Note: have to turn down regression lambda!!
IAI.fit!(lnr, nX, Y)

In [ ]:
IAI.write_json("lnr.json", lnr)

In [ ]:
nlnr = IAI.read_json("lnr.json")

In [ ]:
# Plotting actual vs. predictions
using Statistics
using Plots
Ypredict = IAI.predict(lnr, nX)
leaf_index, leaves = bin_to_leaves(lnr, nX)
display(Plots.plot(X[:,2], X[:,1], Ypredict,
     seriestype=:scatter, markersize = 2, zcolor=leaf_index, seriescolor=:phase, camera=(10,50), legend=false));
print("MSE: ", mean((Ypredict-Y).^2))

In [ ]:
# Deep trees